# 🎫 Ticket Initial Responder - Google Colab Notebook

## Technical Overview: Agentic RAG for Ticket Response

This notebook demonstrates **Agentic Retrieval-Augmented Generation (RAG)** applied to an intelligent ticket response system.

### What is Agentic RAG?

**Agentic RAG** combines three powerful AI concepts:

1. **Retrieval-Augmented Generation (RAG)**: Enhances LLM responses by retrieving relevant information from knowledge bases before generating answers
2. **Agentic Systems**: Uses AI agents that can make decisions, use tools, and take actions autonomously
3. **Tool-Using Agents**: Agents that can classify, search, and synthesize information using specialized tools

### Our Use Case: Ticket Initial Responder

**Problem**: Support teams receive many tickets. Many can be answered from existing knowledge bases (FAQs), but manual routing and response is time-consuming.

**Solution**: An intelligent agent that:
- **Classifies** incoming tickets automatically (DB, DevOps, or Other)
- **Retrieves** relevant information from domain-specific FAQ knowledge bases
- **Generates** professional, contextual responses
- **Falls back** gracefully when no FAQ match is found

### Technical Architecture

```
┌─────────────────┐
│  Incoming Ticket│
└────────┬────────┘
         │
         ▼
┌─────────────────────────┐
│  Classification Tool    │  ← Determines: DB, DevOps, or Other
│  (Keyword-based)        │
└────────┬────────────────┘
         │
         ├─── DB ────────► ┌──────────────────┐
         │                 │  DB FAQ Vector   │
         │                 │  Store (FAISS)   │
         │                 └────────┬─────────┘
         │                          │
         ├─── DevOps ────► ┌────────┴─────────┐
         │                 │ DevOps FAQ Vector│
         │                 │ Store (FAISS)    │
         │                 └────────┬─────────┘
         │                          │
         └─── Other ──────► ┌───────┴────────┐
                            │ Fallback Tool  │
                            └───────┬────────┘
                                    │
                                    ▼
                            ┌───────────────┐
                            │  LLM Agent    │  ← Synthesizes response
                            │  (Groq/GPT)   │
                            └───────┬───────┘
                                    │
                                    ▼
                            ┌───────────────┐
                            │  Final        │
                            │  Response     │
                            └───────────────┘
```

### Key Components

1. **Vector Stores (FAISS)**: 
   - Stores FAQ entries as embeddings for semantic search
   - Separate stores for DB and DevOps FAQs
   - Enables finding relevant answers even with different wording

2. **Embeddings (Sentence Transformers)**:
   - Converts text to numerical vectors
   - Model: `all-MiniLM-L6-v2` (lightweight, fast, accurate)

3. **LLM Agent (Groq)**:
   - Orchestrates the workflow
   - Uses tools to classify and search
   - Synthesizes final responses

4. **Tools**:
   - `classify_ticket`: Routes tickets to correct domain
   - `check_db_faq`: Searches database FAQ knowledge base
   - `check_devops_faq`: Searches DevOps FAQ knowledge base
   - `generate_fallback_message`: Creates helpful responses when no FAQ match

### Why This Approach?

- **Efficiency**: Automates initial ticket triage and response
- **Accuracy**: Uses semantic search to find relevant answers
- **Scalability**: Can handle multiple FAQ categories
- **Flexibility**: Agent decides which tools to use based on ticket content
- **Reliability**: Falls back gracefully when no answer is found

---

**Follow the cells in order to build and test this system step-by-step!** 🚀

## Step 1: Install Dependencies

**What this cell does:**
- Installs all required Python packages for the ticket responder system
- Uses `pip` with the `-q` flag for quiet installation (less output)

**Packages installed:**
- `langchain-core`, `langchain-community`, `langchain`: Core LangChain framework for building LLM applications
- `langchain-huggingface`: Integration for HuggingFace embeddings (sentence transformers)
- `langchain-groq`: Integration for Groq LLM API (fast inference)
- `faiss-cpu`: Facebook AI Similarity Search library for efficient vector similarity search
- `sentence-transformers`: Pre-trained models for converting text to embeddings
- `python-dotenv`: For loading environment variables from `.env` files
- `typing-extensions`: Additional type hints for Python

**Why these packages?**
- LangChain provides the agent framework and tool system
- FAISS enables fast semantic search across FAQ entries
- Sentence transformers create embeddings for similarity matching
- Groq provides fast LLM inference for the agent

**Run this cell first** - it may take 1-2 minutes to download and install all packages.

In [ ]:
!pip install -q langchain-core langchain-community langchain-huggingface langchain-groq langchain faiss-cpu sentence-transformers python-dotenv typing-extensions

## Step 2: Set Up Your Groq API Key

**What this cell does:**
- Configures authentication for the Groq LLM API
- Stores the API key in environment variables for secure access
- The agent will use this key to make API calls to Groq's language models

**Why Groq?**
- Fast inference speed (optimized hardware)
- Cost-effective for high-volume use cases
- Supports various open-source models (we use `gpt-oss-20b`)

**Get your API key from:** https://console.groq.com/keys

**Option A: Using Colab Secrets (Recommended - More Secure)**
1. Click the 🔑 icon in the left sidebar
2. Click "Add new secret"
3. Name: `GROQ_API_KEY`
4. Value: Your API key from Groq console
5. Use the code in the next cell

**Option B: Direct Input (Less Secure - For Testing)**
- Use the alternative cell below to enter your key directly
- Note: This will be visible in the notebook output

**Security Note:** Never commit API keys to version control. Colab Secrets are encrypted and not visible in notebook outputs.

In [ ]:
# Option A: Using Colab Secrets (Recommended)
from google.colab import userdata
import os

try:
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
    print("✅ API key loaded from Colab secrets")
except Exception as e:
    print(f"⚠️ Secret not found: {e}")
    print("Please add GROQ_API_KEY to Colab secrets or use Option B below")

In [ ]:
# Option B: Direct Input (Alternative)
# Uncomment and run this if you didn't use Colab secrets
# import os
# api_key = input("Enter your Groq API key: ")
# os.environ['GROQ_API_KEY'] = api_key
# print("✅ API key set")

## Step 3: Upload FAQ CSV Files

Upload your `db_faq.csv` and `devops_faq.csv` files.

**Format:** CSV files with columns: `question`, `answer`

**Alternative:** If you don't have files, we'll create sample ones for you in the next cell.

In [ ]:
from google.colab import files
import os

print("Upload db_faq.csv file:")
uploaded_db = files.upload()

print("\nUpload devops_faq.csv file:")
uploaded_devops = files.upload()

# Rename files if needed
for filename in uploaded_db.keys():
    if filename != 'db_faq.csv':
        os.rename(filename, 'db_faq.csv')
        print(f"Renamed {filename} to db_faq.csv")

for filename in uploaded_devops.keys():
    if filename != 'devops_faq.csv':
        os.rename(filename, 'devops_faq.csv')
        print(f"Renamed {filename} to devops_faq.csv")

print("\n✅ Files uploaded successfully!")

### Alternative: Create Sample FAQ Files

If you don't have FAQ files, run this cell to create sample ones.

In [ ]:
import csv

# Create sample DB FAQ
db_faq_data = [
    ["question", "answer"],
    ["How do I reset a database connection pool?", "To reset a database connection pool, you can restart the application server or use the connection pool management interface. For most connection pools (HikariCP, C3P0, etc.), you can call the close() method on the DataSource and recreate it. In production, coordinate with the team to avoid service disruption."],
    ["What causes database connection timeout errors?", "Database connection timeout errors typically occur due to: 1) Network issues between application and database, 2) Database server being overloaded, 3) Connection pool exhaustion, 4) Firewall or security group blocking connections, 5) Incorrect connection string or credentials. Check network connectivity, database server health, and connection pool settings."],
    ["How do I optimize a slow SQL query?", "To optimize a slow SQL query: 1) Use EXPLAIN or EXPLAIN ANALYZE to understand the execution plan, 2) Add appropriate indexes on columns used in WHERE, JOIN, and ORDER BY clauses, 3) Avoid SELECT * and only fetch needed columns, 4) Use query hints if necessary, 5) Consider partitioning large tables, 6) Update table statistics. Always test changes in a non-production environment first."],
    ["How do I check database connection health?", "To check database connection health: 1) Run a simple query like SELECT 1, 2) Check connection pool metrics (active/idle connections), 3) Monitor connection timeout errors in logs, 4) Use database-specific health check commands, 5) Verify network connectivity (ping, telnet to database port), 6) Check database server resource usage (CPU, memory, connections)."],
    ["What is connection pooling and why is it important?", "Connection pooling is a technique that maintains a cache of database connections that can be reused across multiple requests. It's important because: 1) Creating connections is expensive, 2) It limits the number of concurrent connections to the database, 3) It improves application performance, 4) It helps prevent connection exhaustion. Configure pool size based on your application's concurrency needs."]
]

with open('db_faq.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(db_faq_data)

# Create sample DevOps FAQ
devops_faq_data = [
    ["question", "answer"],
    ["How do I deploy an application to production?", "To deploy to production: 1) Ensure all tests pass in staging, 2) Create a deployment plan with rollback steps, 3) Notify stakeholders of deployment window, 4) Use CI/CD pipeline for automated deployment, 5) Monitor application health during and after deployment, 6) Verify critical functionality, 7) Keep deployment logs for troubleshooting. Always deploy during low-traffic periods when possible and have a rollback plan ready."],
    ["What causes deployment failures?", "Deployment failures can be caused by: 1) Configuration errors (environment variables, secrets), 2) Dependency issues (missing packages, version conflicts), 3) Resource constraints (memory, CPU limits), 4) Network connectivity issues, 5) Database migration failures, 6) Health check failures, 7) Insufficient permissions, 8) Container image issues. Check deployment logs, verify configuration, and test in staging first."],
    ["How do I troubleshoot a failed Kubernetes pod?", "To troubleshoot a failed pod: 1) Check pod status with 'kubectl get pods', 2) Describe the pod for events: 'kubectl describe pod <pod-name>', 3) Check logs: 'kubectl logs <pod-name>', 4) Verify resource limits and requests, 5) Check image pull issues, 6) Verify configuration maps and secrets, 7) Check node resources, 8) Review pod events for scheduling issues. Common issues: OOMKilled, ImagePullBackOff, CrashLoopBackOff."],
    ["How do I scale an application horizontally?", "To scale horizontally: 1) Ensure application is stateless or uses shared state storage, 2) Configure load balancer to distribute traffic, 3) Use auto-scaling groups (Kubernetes HPA, AWS ASG), 4) Set appropriate scaling metrics (CPU, memory, request rate), 5) Test scaling behavior under load, 6) Monitor resource usage, 7) Configure health checks. Horizontal scaling adds more instances, while vertical scaling increases instance size."],
    ["What is container orchestration and why is it needed?", "Container orchestration automates deployment, scaling, and management of containerized applications. It's needed because: 1) Manages multiple containers across multiple hosts, 2) Handles service discovery and load balancing, 3) Provides self-healing capabilities, 4) Enables rolling updates and rollbacks, 5) Manages resource allocation. Popular tools: Kubernetes, Docker Swarm, Amazon ECS, Nomad."]
]

with open('devops_faq.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(devops_faq_data)

print("✅ Sample FAQ files created!")
print(f"📄 db_faq.csv: {len(db_faq_data)-1} entries")
print(f"📄 devops_faq.csv: {len(devops_faq_data)-1} entries")

## Step 4: Load and Initialize the Ticket Responder

This cell loads the FAQ databases and initializes the ticket responder system.

In [ ]:
import csv
import os
import logging
from typing import List

from langchain_core.documents import Document
from langchain_core.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain.agents import create_agent

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration
SIMILARITY_THRESHOLD = 0.7
MAX_FAQ_RESULTS = 3

def load_faq_csv(path: str, faq_type: str = "general") -> List[Document]:
    """Load FAQ entries from a CSV file."""
    docs = []
    if not os.path.exists(path):
        logger.warning(f"FAQ file not found: {path}")
        return docs
    
    try:
        with open(path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                q = row.get("question", "").strip()
                a = row.get("answer", "").strip()
                if q and a:
                    docs.append(Document(
                        page_content=f"Q: {q}\nA: {a}",
                        metadata={"faq_type": faq_type, "question": q, "answer": a}
                    ))
        logger.info(f"Loaded {len(docs)} FAQ entries from {path}")
    except Exception as e:
        logger.error(f"Error loading FAQ from {path}: {e}")
    
    return docs

print("📚 Loading FAQ databases...")

# Load FAQ databases
db_docs = load_faq_csv("./db_faq.csv", faq_type="db")
devops_docs = load_faq_csv("./devops_faq.csv", faq_type="devops")

# Initialize embeddings
print("🔤 Initializing embeddings...")
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

# Create text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Create separate vector stores
db_store = None
devops_store = None

if db_docs:
    print(f"📊 Creating DB FAQ vector store ({len(db_docs)} entries)...")
    db_chunks = splitter.split_documents(db_docs)
    db_store = FAISS.from_documents(db_chunks, emb)
    print(f"✅ DB FAQ vector store created with {len(db_chunks)} chunks")

if devops_docs:
    print(f"📊 Creating DevOps FAQ vector store ({len(devops_docs)} entries)...")
    devops_chunks = splitter.split_documents(devops_docs)
    devops_store = FAISS.from_documents(devops_chunks, emb)
    print(f"✅ DevOps FAQ vector store created with {len(devops_chunks)} chunks")

print("\n✅ FAQ databases loaded successfully!")

## Step 5: Define Tools and Agent

This cell defines the classification, FAQ search, and fallback tools, then creates the agent.

In [ ]:
@tool
def classify_ticket(ticket_content: str) -> str:
    """Classify a ticket to determine if it's related to database, DevOps, or other."""
    ticket_lower = ticket_content.lower()
    
    # Database-related keywords
    db_keywords = [
        "database", "db", "sql", "query", "table", "schema", "migration",
        "connection", "connection pool", "transaction", "index", "constraint",
        "postgres", "mysql", "oracle", "mongodb", "redis", "dynamodb",
        "rds", "data", "record", "row", "column", "primary key", "foreign key"
    ]
    
    # DevOps-related keywords
    devops_keywords = [
        "deployment", "ci/cd", "pipeline", "docker", "kubernetes", "k8s",
        "container", "infrastructure", "aws", "cloud", "terraform", "ansible",
        "jenkins", "gitlab", "github actions", "monitoring", "logging",
        "scaling", "load balancer", "server", "vm", "instance", "ec2",
        "eks", "ecs", "lambda", "s3", "cloudformation", "iam", "vpc"
    ]
    
    db_score = sum(1 for keyword in db_keywords if keyword in ticket_lower)
    devops_score = sum(1 for keyword in devops_keywords if keyword in ticket_lower)
    
    if db_score > devops_score and db_score > 0:
        return "db"
    elif devops_score > db_score and devops_score > 0:
        return "devops"
    else:
        return "other"

@tool
def check_db_faq(ticket_content: str) -> str:
    """Search the Database FAQ knowledge base for relevant information."""
    if db_store is None:
        return "DB FAQ database is not available."
    
    try:
        results = db_store.similarity_search_with_score(ticket_content, k=MAX_FAQ_RESULTS)
        
        if not results:
            return "No relevant DB FAQ entries found for this ticket."
        
        filtered_results = [(doc, score) for doc, score in results if score >= SIMILARITY_THRESHOLD]
        
        if not filtered_results:
            return f"No DB FAQ entries found above similarity threshold ({SIMILARITY_THRESHOLD})."
        
        context = "\n\n---\n\n".join([
            f"DB FAQ Entry {i+1} (Relevance: {1-score:.2%}):\n{doc.page_content}" 
            for i, (doc, score) in enumerate(filtered_results)
        ])
        
        return f"Found {len(filtered_results)} relevant DB FAQ entries:\n\n{context}"
    
    except Exception as e:
        return f"Error searching DB FAQ: {str(e)}"

@tool
def check_devops_faq(ticket_content: str) -> str:
    """Search the DevOps FAQ knowledge base for relevant information."""
    if devops_store is None:
        return "DevOps FAQ database is not available."
    
    try:
        results = devops_store.similarity_search_with_score(ticket_content, k=MAX_FAQ_RESULTS)
        
        if not results:
            return "No relevant DevOps FAQ entries found for this ticket."
        
        filtered_results = [(doc, score) for doc, score in results if score >= SIMILARITY_THRESHOLD]
        
        if not filtered_results:
            return f"No DevOps FAQ entries found above similarity threshold ({SIMILARITY_THRESHOLD})."
        
        context = "\n\n---\n\n".join([
            f"DevOps FAQ Entry {i+1} (Relevance: {1-score:.2%}):\n{doc.page_content}" 
            for i, (doc, score) in enumerate(filtered_results)
        ])
        
        return f"Found {len(filtered_results)} relevant DevOps FAQ entries:\n\n{context}"
    
    except Exception as e:
        return f"Error searching DevOps FAQ: {str(e)}"

@tool
def generate_fallback_message(ticket_content: str, classification: str = "other") -> str:
    """Generate a helpful fallback message when no FAQ entries are found."""
    base_message = """Thank you for submitting your ticket. We've received your request and are here to help.

While we couldn't find an immediate answer in our knowledge base, our team will review your ticket and respond shortly.

To help us assist you better, please ensure your ticket includes:
- A clear description of the issue or question
- Any error messages or relevant logs
- Steps you've already taken to resolve the issue
- Expected vs. actual behavior

"""
    
    if classification == "db":
        additional_guidance = """
For database-related issues, it would be helpful if you could also provide:
- Database type and version
- Connection details (if applicable and secure)
- Query examples or schema information
- Any relevant error codes or messages
"""
    elif classification == "devops":
        additional_guidance = """
For DevOps-related issues, it would be helpful if you could also provide:
- Environment details (staging, production, etc.)
- Deployment configuration or pipeline logs
- Infrastructure details (AWS, Kubernetes, etc.)
- Any monitoring or alerting information
"""
    else:
        additional_guidance = """
If you have any additional context, screenshots, or documentation that might help,
please feel free to add them to this ticket.
"""
    
    return base_message + additional_guidance + "\nWe appreciate your patience and will get back to you soon."

# Define tools
tools = [classify_ticket, check_db_faq, check_devops_faq, generate_fallback_message]

# Initialize the language model
print("🤖 Initializing language model...")
model = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    api_key=os.getenv("GROQ_API_KEY")
)

system_prompt = """You are an intelligent ticket initial responder that helps route and respond to support tickets.

Your workflow should be:
1. First, use classify_ticket to determine if the ticket is DB-related, DevOps-related, or other
2. Based on the classification:
   - If "db": Use check_db_faq to search the Database FAQ
   - If "devops": Use check_devops_faq to search the DevOps FAQ
   - If "other" or no relevant FAQ found: Use generate_fallback_message to create a helpful response
3. Synthesize the information and provide a clear, professional response to the user

Guidelines:
- Always start by classifying the ticket
- Use the appropriate FAQ tool based on classification
- If FAQ results are found, incorporate them into your response naturally
- If no relevant FAQ entries are found, use the fallback message tool
- Be empathetic, professional, and helpful in all responses
- Acknowledge the user's issue and provide clear next steps
- If the ticket spans multiple categories, check both FAQs if needed

Think step-by-step and provide the best possible initial response to help the user."""

print("🔧 Creating agent...")
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt
)

print("\n✅ Agent created successfully!")

## Step 6: Test the Ticket Responder

Now let's test the system with sample tickets!

In [ ]:
def test_ticket(ticket_content: str):
    """Test the ticket responder with a sample ticket."""
    print(f"\n{'='*70}")
    print(f"🎫 TICKET: {ticket_content}")
    print(f"{'='*70}\n")
    
    try:
        # Invoke the agent
        result = agent.invoke({
            "messages": [("human", f"Please help me respond to this ticket: {ticket_content}")]
        })
        
        # Get the response
        response = result['messages'][-1].content
        
        print("📝 RESPONSE:")
        print("-" * 70)
        print(response)
        print("-" * 70)
        print(f"\n✅ Successfully processed ticket\n")
        
        return response
    
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Test function ready!")

### Test Case 1: Database-Related Ticket

In [ ]:
test_ticket("My database connection is timing out. How do I fix this?")

### Test Case 2: DevOps-Related Ticket

In [ ]:
test_ticket("My Kubernetes pod is in CrashLoopBackOff state. What should I do?")

### Test Case 3: General Ticket

In [ ]:
test_ticket("I need help with my application deployment")

## Step 7: Test Your Own Tickets

Enter your own tickets below to test the system!

In [ ]:
# Enter your ticket here
your_ticket = "How do I optimize a slow SQL query?"

test_ticket(your_ticket)

## Step 8: Batch Testing

Test multiple tickets at once!

In [ ]:
test_tickets = [
    "How do I reset a database connection pool?",
    "What causes deployment failures?",
    "How do I scale an application horizontally?",
    "What is connection pooling and why is it important?",
    "How do I troubleshoot a failed Kubernetes pod?"
]

print(f"🚀 Running batch test with {len(test_tickets)} tickets...\n")

for i, ticket in enumerate(test_tickets, 1):
    print(f"\n[Test {i}/{len(test_tickets)}]")
    test_ticket(ticket)

## 🎉 Summary

You've successfully set up and tested the Ticket Initial Responder!

**What you've accomplished:**
- ✅ Installed all dependencies
- ✅ Configured API keys
- ✅ Loaded FAQ databases
- ✅ Created vector stores for semantic search
- ✅ Tested the system with various tickets

**Next Steps:**
- Customize the FAQ CSV files with your own entries
- Adjust similarity threshold if needed
- Deploy to production using AWS Bedrock AgentCore

**Tips:**
- The system automatically classifies tickets
- It searches relevant FAQ databases based on classification
- Falls back to helpful messages when no FAQ match is found
- You can add more FAQ categories by creating additional CSV files

Happy ticket responding! 🎫✨